<a href="https://colab.research.google.com/github/aborbala/tree-canopy/blob/main/01_01_DOP_slicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script for Slicing Digital Orthophoto (DOP) Data

This script processes Digital Orthophoto (DOP) data, converting 2km x 2km tiles into 100m x 100m slices.
The reason for this slicing is to match the scale of the LiDAR (LAS) data, which is also in 1km x 1km tiles.

Steps:
1. Load and preprocess the DOP data into 1km x 1km tiles (originally in 2km x 2km tiles).
2. Slice the 1km x 1km DOP data into 100m x 100m segments for efficient management and analysis.

Parameters:
    dop_data (str): Path to the 1km x 1km preprocessed DOP data.

Returns:
    list: Paths to the processed 100m x 100m DOP slices.

Example Usage:
    python slice_dop_data.py --dop_data path_to_dop_data

Author:
    Your Name
    Date


In [ ]:
! pip install --upgrade rasterio laspy -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.0 MB/s eta 0:00:00


In [ ]:
import rasterio
import numpy as np
import os
import laspy
from rasterio.windows import Window
from rasterio.plot import show
import ipdb

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Load helper functions
#!ls /content/drive/MyDrive/utils/*.py
#!cat '/content/drive/My Drive/utils/utils.py'
#import sys
#sys.path.append('/content/drive/MyDrive/utils/')
#import utils
#utils.test()

In [ ]:
def get_las_bbox(file_path):
    """
    Get the bounding box coordinates from a LAS file.

    Args:
        file_path (str): Path to the LAS file.

    Returns:
        tuple: (min_x, min_y, max_x, max_y) coordinates of the bounding box.
    """
    las = laspy.read(file_path)
    min_x = las.header.min[0]
    min_y = las.header.min[1]
    max_x = las.header.max[0]
    max_y = las.header.max[1]
    return min_x, min_y, max_x, max_y

In [ ]:
def convert_to_rgb_tiff(input_image_path, output_image_path, num_bands=3):
   """
    Convert an input image to an RGB TIFF imagee.

    Args:
        input_image_path (str): Path to the input image file.
        output_image_path (str): Path to the output RGB TIFF file.
        num_bands (int, optional): Number of bands to include in the output image. Defaults to 3.

    Returns:
        None
    """
    with rasterio.open(input_image_path) as src:
        profile = src.profile
        image_data = src.read()

    # Select the first three bands
    rgb_image_data = image_data[0:3, :, :]

    # Update the profile for the output image
    profile.update(count=3)

    # Write the RGB image to a new TIFF file
    with rasterio.open(output_image_path, 'w', **profile) as dst:
        dst.write(rgb_image_data)

In [ ]:
def ensure_rgb_tiff(input_image_path, output_image_path=None, num_bands=3):
     """
    Ensure the input image is an RGB TIFF with the specified number of bands for PIL package.

    Args:
        input_image_path (str): Path to the input image file.
        output_image_path (str, optional): Path to the output RGB TIFF file. Defaults to None.
        num_bands (int, optional): Number of bands to include in the output image. Defaults to 3.

    Returns:
        str: Path to the output RGB TIFF file if conversion is needed, otherwise the input image path.
    """
    with rasterio.open(input_image_path) as src:
        image_data = src.read()

    # Check if the input image has more bands than the specified 'num_bands'
    if image_data.shape[0] > num_bands:
        if output_image_path is None:
            base, ext = os.path.splitext(input_image_path)
            output_image_path = f"{base}_rgb{ext}"

        # Call the 'convert_to_rgb_tiff' function to convert the input image
        convert_to_rgb_tiff(input_image_path, output_image_path, num_bands)
        return output_image_path
    else:
        return input_image_path

In [ ]:
def crop_tif_by_bbox(tif_file_path, bbox):
    """
    Crop a TIFF image using a bounding box (LAS) to ensure compatibility.

    Args:
        tif_file_path (str): Path to the TIFF image file.
        bbox (tuple): Bounding box coordinates (min_x, min_y, max_x, max_y).

    Returns:
        tuple: Cropped image data and updated profile.
    """
    with rasterio.open(tif_file_path) as src:
        min_x, min_y, max_x, max_y = bbox
        window = src.window(min_x, min_y, max_x, max_y)
        transform = src.window_transform(window)
        profile = src.profile
        profile.update({
            'height': window.height,
            'width': window.width,
            'transform': transform
        })
        data = src.read(window=window)

        # Ensure the data is in RGB format
        if data.shape[0] > 3:
            data = data[0:3, :, :]
            profile.update(count=3)

    return data, profile


In [ ]:
def slice_image(input_data, input_profile, output_folder, tilename, div=10):
    """
    Slice an image into smaller tiles.

    Args:
        input_data (numpy.ndarray): Image data array.
        input_profile (dict): Raster profile containing metadata about the image.
        output_folder (str): Folder to save the sliced image tiles.
        tilename (str): Base name for the output tiles.
        div (int, optional): Number of divisions along each dimension. Defaults to 10.

    Returns:
        None
    """
    rows, cols = input_data.shape[1], input_data.shape[2]
    transform = input_profile['transform']
    profile = input_profile.copy()

    xsize = cols // div
    ysize = rows // div

    for i in range(div):
        for j in range(div):
            row_min, row_max = j * ysize, (j + 1) * ysize
            col_min, col_max = i * xsize, (i + 1) * xsize

            window_transform = rasterio.windows.transform(
                rasterio.windows.Window(col_min, row_min, col_max - col_min, row_max - row_min),
                transform)

            profile.update({
                'height': row_max - row_min,
                'width': col_max - col_min,
                'transform': window_transform
            })

            output_data = input_data[:, row_min:row_max, col_min:col_max]

            if np.all(output_data == input_profile['nodata']):
                continue

            output_file_path = os.path.join(output_folder, f"{tilename}_{i}_{j}.tif")
            print(output_file_path)

            with rasterio.open(output_file_path, 'w', **profile) as dst:
                dst.write(output_data)


In [ ]:
# Path to the folder containing LAS files without buildings
#las_folder_path = '/content/drive/MyDrive/data/382_5826_1/LAS_no_buildings'
las_folder_path = '/content/drive/MyDrive/data/400_5816/LAS_no_buildings'

## Summer 2020
#tif_file_path = '/content/drive/MyDrive/data/382_5826_1/DOP/truedop20rgb_382_5826_2_be_2020.tif'
#sliced_output_folder_path = '/content/drive/MyDrive/data/382_5826_1/sliced_output_2020S'

# Paths to the TIFF files for different time periods and locations
## Winter 02.2021 382_5826
#tif_file_path = '/content/drive/MyDrive/data/382_5826_1/DOP/dop20rgbi_33_382_5826_2_be_2022.tif'
#sliced_output_folder_path = '/content/drive/MyDrive/data/382_5826_1/sliced_output_2021W'

## Summer 2020 400_5816
tif_file_path = '/content/drive/MyDrive/data/400_5816/DOP/dop20rgb_400_5816_2_be_2020.tif'
sliced_output_folder_path = '/content/drive/MyDrive/data/400_5816/sliced_output_2020S'

os.makedirs(sliced_output_folder_path, exist_ok=True)

# Iterate over each file in the LAS folder
for file in os.listdir(las_folder_path):
    print(file)
    if file.endswith('.las'):
        las_file_path = os.path.join(las_folder_path, file)
        bbox = get_las_bbox(las_file_path)
        print(las_file_path)

        # Call the crop_tif_by_bbox function and get the cropped data and profile
        cropped_data, cropped_profile = crop_tif_by_bbox(tif_file_path, bbox)

        # Call the slice_image function for the cropped data and profile
        tilename = os.path.splitext(file)[0]
        slice_image(cropped_data, cropped_profile, sliced_output_folder_path, tilename)

3dm_33_401_5817_1_be_nobuild.las
/content/drive/MyDrive/data/400_5816/LAS_no_buildings/3dm_33_401_5817_1_be_nobuild.las
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_0.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_1.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_2.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_3.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_4.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_5.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_6.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_7.tif
/content/drive/MyDrive/data/400_5816/sliced_output_2020S/3dm_33_401_5817_1_be_nobuild_0_8.tif
/content/drive/MyDrive/data/400_58

In [ ]:
# Slice the tile: 33_382_5826_1 (1km x 1km)
#input_image_path = "/content/drive/MyDrive/data/382_5826_1/dop20rgbi_33_382_5826_2_be_2021_1.tif" # 2021W
#rgb_image_path = '/content/drive/MyDrive/data/382_5826_1/dop20rgbi_33_382_5826_2_be_2021_1_rgb.tif'

In [ ]:
# If the input image has more bands than 'num_bands', the function returns the path to the new RGB image.
# Otherwise, it returns the input image path.
#result_image_path = ensure_rgb_tiff(input_image_path, rgb_image_path)

In [ ]:
#utils.sliceImg(result_image_path, "/content/drive/MyDrive/data/382_5826_1/sliced_imgs_2021W/", "382_5826")

5000 5000


## Now go to R and extract the tree crowns for each tile